<h1 style="text-align: center; font-size: 50px;"> Code Generation RAG with Langchain </h1>

This notebook uses the **RAG (Retrieval-Augmented Generation)** technique to enhance code generation using context-aware prompts. It extracts code and documentation from GitHub repositories (including Python files, Jupyter notebooks, and other programming languages), transforms them into vector embeddings, and stores them in a vector database. When a user submits a prompt, the system retrieves the most relevant code snippets and provides them as context to a language model (LLM), which then generates new code based on that context.

## Notebook Overview

- Start Execution
- Install and Import Libraries
- Configure Settings
- Cloning and Extracting Code from Github Repositories
- Generating Metadata with LLM
- Generate Embeddings and Structure Data
- Store and Query Documents in ChromaDB
- Code Generation Chain


# Start Execution

In [1]:
import logging
import time

# Configure logger
logger: logging.Logger = logging.getLogger("run_workflow_logger")
logger.setLevel(logging.INFO)
logger.propagate = False  # Prevent duplicate logs from parent loggers

# Set formatter
formatter: logging.Formatter = logging.Formatter(
    fmt="%(asctime)s - %(levelname)s - %(message)s",
    datefmt="%Y-%m-%d %H:%M:%S"
)

# Configure and attach stream handler
stream_handler: logging.StreamHandler = logging.StreamHandler()
stream_handler.setFormatter(formatter)
logger.addHandler(stream_handler)

In [2]:
start_time = time.time()  

logger.info("Notebook execution started.")

2025-08-05 20:21:55 - INFO - Notebook execution started.


# Install and Import Libraries

In this step, we import all the necessary libraries and internal components required to run the RAG pipeline, including modules for notebook parsing, embedding generation, vector storage, and code generation with LLMs.


In [3]:
%%time

%pip install -r ../requirements.txt --quiet

Note: you may need to restart the kernel to use updated packages.
CPU times: user 15.6 ms, sys: 18.4 ms, total: 34 ms
Wall time: 1.81 s


In [4]:
# === Built-in ===
import os
import sys
import datetime
from pathlib import Path
from typing import List

# === Third-party libraries ===
import pandas as pd
import warnings
import yaml
from IPython import get_ipython
from IPython.display import display, Markdown, Code

# === Langchain ===
from langchain.prompts import ChatPromptTemplate
from langchain.schema import Document, StrOutputParser
from langchain.schema.runnable import RunnablePassthrough
from langchain_core.prompts import PromptTemplate
from langchain_huggingface import HuggingFaceEmbeddings

# === Internal modules ===

# Add 'src' directory to system path (2 levels up)
src_path = os.path.abspath(os.path.join(os.getcwd(), ".."))
if src_path not in sys.path:
    sys.path.append(src_path)
 
# Utils
from src.utils import (
    load_config,
    load_secrets,
    load_secrets_to_env,
    configure_proxy,
    initialize_llm,
    configure_hf_cache,
    clean_code,
    generate_code_with_retries,
    get_context_window,
    dynamic_retriever,
    format_docs_with_adaptive_context
)

# Core components
from core.extract_text.github_repository_extractor import GitHubRepositoryExtractor
from core.generate_metadata.llm_context_updater import LLMContextUpdater
from core.dataflow.dataflow import EmbeddingUpdater, DataFrameConverter
from core.vector_database.vector_store_writer import VectorStoreWriter
from core.extract_text.rag_utils import (
    identify_question_type,
    retriever,
    format_multi_doc_context,
    process_repository_question
)
from core.prompt_templates import (
    get_dynamic_repository_prompt,
    get_code_description_prompt,
    get_code_generation_prompt,
    get_specialized_prompt,
    get_metadata_generation_prompt
)

# Configure Settings

In [5]:
# Suppress Python warnings
warnings.filterwarnings("ignore")

In [6]:
CONFIG_PATH = "../configs/config.yaml"
SECRETS_PATH = "../configs/secrets.yaml"
MLFLOW_EXPERIMENT_NAME = "Code-Generation-Experiment"
MLFLOW_RUN_NAME = "Code-Generation-Run"
LOCAL_MODEL_PATH = "/home/jovyan/datafabric/meta-llama3.1-8b-Q8/Meta-Llama-3.1-8B-Instruct-Q8_0.gguf"
MLFLOW_MODEL_NAME = "Code-Generation-Model"

## Configuration and Secrets Loading

In this section, we load configuration parameters and API keys from separate YAML files. This separation helps maintain security by keeping sensitive information (API keys) separate from configuration settings.

- **config.yaml**: Contains non-sensitive configuration parameters like model sources and URLs
- **secrets.yaml**: Contains sensitive API keys for services like HuggingFace
- *(Optional for Premium users)* Secrets such as API keys for services like HuggingFace can be stored as environment variables for the project and loaded into the notebook (see the project's README file for steps on how to save secrets in Secrets Manager).

In [7]:
# Load secrets from secrets.yaml file (if it exists) into environment
if Path(SECRETS_PATH).exists():
    load_secrets_to_env(SECRETS_PATH)
else:
    print(f"No secrets file found at {SECRETS_PATH}; relying on preexisting environment")

# Retrieve secrets from environment
try:
    secrets = load_secrets()
except ValueError:
    secrets = {}

# Load configuration and secrets
config = load_config(CONFIG_PATH)

print("✅ Configuration loaded successfully")
print("✅ Secrets loaded successfully")

No secrets file found at ../configs/secrets.yaml; relying on preexisting environment
✅ Configuration loaded successfully
✅ Secrets loaded successfully


### Environment Configuration
The following cell configures any necessary environment variables for the workflow.

In [8]:
# Configure environment settings if needed
configure_proxy(config)

### Configuration of Hugging face caches

In the next cell, we configure HuggingFace cache, so that all the models downloaded from them are persisted locally, even after the workspace is closed. This is a future desired feature for AI Studio and the GenAI addon.

In [9]:
# Configure HuggingFace cache
configure_hf_cache()

In [10]:
# Initialize HuggingFace Embeddings
embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

INFO:datasets:PyTorch version 2.7.1 available.
INFO:sentence_transformers.SentenceTransformer:Use pytorch device_name: cuda
INFO:sentence_transformers.SentenceTransformer:Load pretrained SentenceTransformer: all-MiniLM-L6-v2


## Step 1: Cloning and Extracting Code from GitHub Repositories

In this step, we clone a GitHub repository, locate relevant code files, and extract code snippets along with their documentation context.

Using the `GitHubReposito
ryExtractor`, the process includes:
- Cloning the repository.
- Recursively searching for supported code files (Python, Jupyter notebooks, JavaScript, etc.).
- Extracting code snippets along with their documentation context.
- Structuring the extracted data with fields like `id`, `code`, `context`, `filename`, and a placeholder for `embedding`.

Our optimized extractor also handles context window overflow issues:
- Limits the maximum file size to 500KB to skip very large files
- Breaks large files into chunks of 100 lines for easier processing
- Uses pattern-based exclusion for minified/bundled files that would exceed token limits
- Provides detailed context information with line numbers for chunked files

In [11]:
%%time

extractor = GitHubRepositoryExtractor(
    repo_url="https://github.com/MunGell/awesome-for-beginners",
    save_dir="./repo_files",
    verbose=True,
    max_file_size_kb=500,
    max_chunk_size=100,
    supported_extensions=('.py', '.ipynb', '.md', '.txt', '.json', '.js', '.ts')
)
extracted_data = extractor.run()

[INFO] Processing repository: MunGell/awesome-for-beginners
[INFO] Removed existing directory: ./repo_files
[INFO] Created directory: ./repo_files
[INFO] Downloaded: ./repo_files/.github/scripts/render-readme.py
[INFO] Downloaded: ./repo_files/CONTRIBUTING.md
[INFO] Downloaded: ./repo_files/README.md
[INFO] Downloaded: ./repo_files/data.json
[INFO] Extracted 30 code snippets from repository


CPU times: user 43.7 ms, sys: 9.42 ms, total: 53.1 ms
Wall time: 3.06 s


## Step 2: Generating Metadata with LLM 🔢

In this step, we use a language model (LLM) to generate concise explanations for each extracted code snippet, enriching the original data with human-readable context.

### What Happens:

- A prompt template is defined to guide the LLM in describing what each code snippet does, using the code, file name, and optional context.
- A `PromptTemplate` object is built from this structure.
- The selected model (e.g., Meta LLaMA 3.1 8B) is initialized using `initialize_llm`.
- The function `update_context_with_llm` runs the model for each code snippet and updates the `context` field with the generated explanation.

This process transforms raw code into meaningful metadata, which improves downstream tasks like search, summarization, or generation.


In [12]:
%%time

if "model_source" in config:
    model_source = config["model_source"]

# Get the metadata generation prompt with model-specific formatting
from core.prompt_templates import get_metadata_generation_prompt
metadata_prompt = get_metadata_generation_prompt(model_source)

llm = initialize_llm(model_source, secrets, LOCAL_MODEL_PATH)

# Create the LLM chain with the metadata prompt
llm_chain = metadata_prompt | llm

CPU times: user 887 ms, sys: 2.06 s, total: 2.95 s
Wall time: 50.9 s


### Generate Metadata with Local LLM

⚠️ Generating metadata using a local language model may be time-consuming.  
Whenever possible, we recommend using a hosted API for faster responses and better performance.

Note: The quality of the generated metadata may be lower when using quantized or compact local models.


In [13]:
%%time

updater = LLMContextUpdater(
    llm_chain=llm_chain,
    prompt_template=metadata_prompt.template if hasattr(metadata_prompt, 'template') else str(metadata_prompt),
    verbose=False,  # Set to True to enable detailed execution logs
    print_prompt=False  # Set to True to display the generated prompt before inference
)

updated_data = updater.update(extracted_data)

Updating Contexts: 100%|██████████| 30/30 [02:33<00:00,  5.13s/it]

CPU times: user 2min 28s, sys: 4.08 s, total: 2min 32s
Wall time: 2min 33s


## Step 3: Generate Embeddings and Structure Data

In this step, we generate semantic embeddings for each code snippet's context and structure the results into a Pandas DataFrame for further processing.

### What Happens:

- **Embedding Generation**  
  We use the HuggingFace model `"all-MiniLM-L6-v2"` to convert each snippet's context into an embedding vector.  
  The `EmbeddingUpdater` class handles this process, updating the `embedding` field for each item in the data structure.

- **Data Structuring**  
  The `DataFrameConverter` class is then used to convert the enriched data into a standardized format.  
  Each entry includes:
  - `id`: Unique identifier
  - `embedding`: Vector representation of the context
  - `code`: Extracted code
  - `metadata`: Including filename and updated context

- **DataFrame Creation**  
  The structured data is converted into a Pandas DataFrame, making it easier to visualize, manipulate, and persist for downstream use.


In [14]:
updater = EmbeddingUpdater(embedding_model=embeddings, verbose=False)
updated_data = updater.update(updated_data)

In [15]:
converter = DataFrameConverter(verbose=False)
df = converter.to_dataframe(updated_data)
# Summary of processed snippets
print(f"Processed {len(df)} snippets from repository.")

Processed 30 snippets from repository.


In [16]:
df

,ids,embeddings,code,metadatas
0,4346f0c0-01c0-4dfb-b18a-ed69405f65d8,"[-0.09054996818304062, 0.039509281516075134, -...","from jinja2 import Environment, FileSystemLoad...",{'filenames': '.github/scripts/render-readme.p...
1,b01064f5-9aea-4558-bd67-10e50cbee8d2,"[-0.11651267111301422, 0.00836348906159401, -0...",# Contribution Guide & Guidelines 🚀\n\nWelcome...,"{'filenames': 'CONTRIBUTING.md', 'context': 'T..."
2,cd622e8d-c2e3-48ef-b820-5379e7e4422b,"[-0.09860657155513763, 0.07316506654024124, -0...",<!-- DO NOT EDIT THIS FILE (`README.md`) -->\n...,"{'filenames': 'README.md', 'context': 'This is..."
3,9f4009c6-a440-451a-88a3-52187d83bca0,"[-0.05564915016293526, -0.026840338483452797, ...",- [Alda](https://github.com/alda-lang/alda) _(...,"{'filenames': 'README.md', 'context': 'Based o..."
4,8be838c6-e548-4add-9240-5033249ab754,"[-0.07446417212486267, -0.0714845284819603, -0...",- [stryker](https://github.com/stryker-mutator...,"{'filenames': 'README.md', 'context': 'The cod..."
5,5c2bc6cc-a00a-456b-a21b-0fe028a2e0dc,"[-0.1429516077041626, 0.01486793253570795, -0....",- [PyMC](https://github.com/pymc-devs/pymc) _(...,"{'filenames': 'README.md', 'context': 'This is..."
6,6f524602-9012-483c-a57f-65834de6477e,"[-0.06581294536590576, 0.09057018160820007, -0...","\nTo the extent possible under law, the author...","{'filenames': 'README.md', 'context': 'This co..."
7,468bea51-be59-412b-88b7-f6ef5e430d44,"[-0.08832647651433945, 0.08632959425449371, -0...","{\n ""sponsors"": [\n {\n ""...","{'filenames': 'data.json', 'context': 'Based o..."
8,7dd101b6-a998-4cf3-b237-9b61e38232ae,"[-0.0731334313750267, 0.0831582173705101, -0.0...","""technologies"": [\n ...","{'filenames': 'data.json', 'context': 'The cod..."
9,e1cf098c-efa8-46c3-a657-c19262a05e0c,"[-0.08989168703556061, 0.06940071284770966, -0...","""technologies"": [\n ...","{'filenames': 'data.json', 'context': 'The pro..."


## Step 4: Store and Query Documents in ChromaDB

In this step, we store the code snippets, metadata, and embeddings in **ChromaDB**, a vector database, and implement a function to query them.

### What Happens:

- Initialize the ChromaDB client and create or retrieve the collection `"my_collection"`.
- Extract `ids`, `documents`, `metadatas`, and `embeddings` from the DataFrame and upsert them into the collection.
- Use the `retriever` function to perform semantic searches and return the most relevant code snippets as `Document` objects.


In [17]:
writer = VectorStoreWriter(collection_name="my_collection", verbose=False)
writer.upsert_dataframe(df)

INFO:chromadb.telemetry.product.posthog:Anonymized telemetry enabled. See                     https://docs.trychroma.com/telemetry for more information.
ERROR:chromadb.telemetry.product.posthog:Failed to send telemetry event ClientStartEvent: capture() takes 1 positional argument but 3 were given
INFO:core.vector_database.vector_store_writer:ChromaDB client initialized with persistent storage at ./chroma_db
ERROR:chromadb.telemetry.product.posthog:Failed to send telemetry event ClientCreateCollectionEvent: capture() takes 1 positional argument but 3 were given
INFO:core.vector_database.vector_store_writer:ChromaDB collection 'my_collection' initialized with persistent storage.
INFO:core.vector_database.vector_store_writer:Upserting 30 documents
INFO:core.vector_database.vector_store_writer:✅ Documents upserted successfully into ChromaDB.
INFO:core.vector_database.vector_store_writer:✅ Documents upserted successfully into ChromaDB.


In [18]:
collection = writer.collection
document_count = writer.collection.count()
print(f"Total documents in the collection: {document_count}")


Total documents in the collection: 30


## Step 5: Code Generation Chain

In this step, we build a LangChain pipeline to generate Python code from natural language questions using context retrieved from ChromaDB.

### What Happens:

- **Context Window Management**  
  We use `get_context_window` to determine the model's token limit, which helps optimize retrieval and formatting.

- **Smart Document Retrieval**  
  The system now automatically adapts the number of documents retrieved based on the context window size:
  - Small contexts (≤4096 tokens): 3 documents
  - Medium contexts (≤8192 tokens): 5 documents  
  - Large contexts (>8192 tokens): 8 documents

- **Accurate Token Estimation**  
  The system uses improved token counting that:
  - Attempts to use the actual model tokenizer when possible
  - Falls back to content-aware estimation (3.2-4.0 chars/token based on content type)
  - Provides better accuracy than the previous fixed 4 chars/token ratio

- **Multi-Layer Context Protection**  
  Context overflow is prevented at multiple levels:
  1. **Smart retrieval**: Fewer docs for smaller context windows
  2. **Intelligent formatting**: The `format_multi_doc_context` uses 75% of context window (vs previous 70%)
  3. **Emergency truncation**: Final safety check with smart break-point detection
  4. **Document structure preservation**: Tries to break at section/paragraph boundaries

- **Build the Chain**  
  The chain takes two inputs: a question and the context retrieved from the vector store.  
  The prompt is passed through the model, and the output is parsed into clean Python code using `StrOutputParser`.

- **Execute and Print Output**  
  The function `clean_and_print_code(result)` cleans up any formatting markers from the model's output and prints the final code.

In [19]:
# Get the code description prompt template
code_description_prompt = get_code_description_prompt(model_source)

# Get the code generation prompt template
code_generation_prompt = get_code_generation_prompt(model_source)

In [20]:
# Initialize the model
model = llm

# Get the context window size of the model for use in retrieval and document formatting
context_window = get_context_window(model)
print(f"Model context window: {context_window} tokens")

Model context window: 4096 tokens


In [21]:
# Function to extract code information from retrieved documents
def extract_code_info_from_docs(inputs):
    # Get retrieval query - standardize on "question" for clarity
    query = inputs.get("question", "")
    if not query:
        query = inputs.get("query", "")
    
    # Add debugging information
    print(f"Searching repository with query: '{query}'")
    
    # Process the repository question with enhanced retrieval and formatting
    # The process_repository_question now has smarter document count selection
    result = process_repository_question(
        query=query,
        collection=collection,
        context_window=context_window,
        top_n=None
    )
    
    print(f"Retrieved {result['document_count']} relevant documents")
    
    if result['document_count'] > 0:
        # Get specialized prompt based on detected question types
        question_types = result.get("question_types", [])
        specialized_prompt = get_specialized_prompt(question_types, model_source)
        
        # The context has already been properly managed by the improved format_multi_doc_context
        context = result["context"]
        
        # Final safety check using the new accurate token estimation
        from src.utils import estimate_tokens_accurate, check_context_fits
        
        fits, estimated_tokens = check_context_fits(
            text=context, 
            context_window=context_window, 
            model=model,
            reserve_tokens=800  # Reserve for prompt template and response
        )
        
        if not fits:
            print(f"Context still too large after processing: {estimated_tokens} tokens (limit: {context_window - 800})")
            # Emergency truncation with better accuracy
            max_chars = int((context_window - 800) * 3.5)  # More accurate estimation
            
            # Smart truncation - try to preserve document structure
            truncation_point = max_chars
            # Look for section breaks first, then paragraph breaks, then sentences
            for break_pattern in ['\n## ', '\n### ', '\n\n', '. ', '\n']:
                last_break = context[:truncation_point].rfind(break_pattern)
                if last_break > truncation_point * 0.8:  # Must retain at least 80% of content
                    truncation_point = last_break + len(break_pattern)
                    break
            
            context = context[:truncation_point] + "\n\n... (truncated to fit context window)"
            
            # Verify the truncation worked
            final_tokens = estimate_tokens_accurate(context, model)
            print(f"Truncated to {final_tokens} tokens")
        else:
            print(f"Context size OK: {estimated_tokens} tokens")
        
        # Return the processed result with specialized prompt
        print(f"✅ Found relevant files with question types: {', '.join(result['question_types'])}")
        return {
            "question": query,
            "code": result.get("primary_code", ""),
            "filename": result.get("primary_filename", ""),
            "context": context,
            "question_types": question_types,
            "specialized_prompt": specialized_prompt
        }
    else:
        # If no documents found, return empty values
        print("❌ No relevant documents found in the repository")
        return {
            "question": query,
            "code": "No code found",
            "filename": "No filename found",
            "context": "No relevant documents retrieved"
        }

# Function to apply specialized prompt template
def apply_specialized_prompt(inputs):
    specialized_prompt = inputs.get("specialized_prompt")
    if specialized_prompt:
        # Use the specialized prompt if available
        return specialized_prompt.format(
            question=inputs["question"],
            context=inputs["context"]
        )
    else:
        # Fall back to default prompt
        return code_description_prompt.format(
            question=inputs["question"],
            context=inputs["context"]
        )

# Create the code description chain with dynamic prompt selection
code_description_chain = extract_code_info_from_docs | code_description_prompt | model | StrOutputParser()

# Create the code generation chain - doesn't need context from repository
code_generation_chain = {
    "question": lambda x: x["question"],
    "context": lambda x: "" 
} | code_generation_prompt | model | StrOutputParser()

In [22]:
def clean_and_print_code(result: str):
    cleaned = clean_code(result)
    print(cleaned)
    
def print_description(result: str):
    print("Code Description:")
    print(result)

In [23]:
# Example usage of the code generation chains

# 1. Set logging level to reduce HTTP request logs
import logging
logging.getLogger("httpx").setLevel(logging.WARNING)

# 2. Define a code description example with a query to retrieve relevant code
code_description_input = {
    "question": "What is the repository about?"
}

# 3. Run the code description chain
description_result = code_description_chain.invoke(code_description_input)

# 4. Print the code description result
print_description(description_result)

# 5. Define a code generation example (no context needed)
code_gen_input = {
    "question": "Write Python code to extract all image URLs from a webpage using BeautifulSoup"
}

# 6. Use the generate_code_with_retries function with our specialized chain
_, clean_code_output = generate_code_with_retries(
    chain=code_generation_chain,
    example_input=code_gen_input,
    max_attempts=3,
    min_code_length=10
)

# 7. Print only the final cleaned code result 
print("\n# Webpage Image URL Scraping Code:")
print(clean_code_output)

INFO:core.extract_text.rag_utils:Question types: ['concept']
INFO:core.extract_text.rag_utils:Expanded query: 'What is the repository about? '


Searching repository with query: 'What is the repository about?'


/home/jovyan/.cache/chroma/onnx_models/all-MiniLM-L6-v2/onnx.tar.gz: 100%|██████████| 79.3M/79.3M [00:04<00:00, 18.1MiB/s]
ERROR:chromadb.telemetry.product.posthog:Failed to send telemetry event CollectionQueryEvent: capture() takes 1 positional argument but 3 were given
INFO:core.extract_text.rag_utils:Retrieved 3 documents after filtering and re-ranking
INFO:core.extract_text.rag_utils:Context budget: 3072 tokens (10752.0 chars)


Retrieved 3 relevant documents
Context still too large after processing: 3358 tokens (limit: 3296)
Truncated to 3143 tokens
✅ Found relevant files with question types: concept
Code Description:
Based on the code context provided, I will attempt to answer your question about what this repository is about.

This repository appears to be a collection of open-source projects across various programming languages and platforms. The repository includes documentation files in Markdown format, as well as JSON and Julia files.

Upon examining the contents of this repository, it seems that the primary focus is on providing a collection of open-source projects for developers to learn from and contribute to.

To answer your specific question about what the repository is about, I would say that it appears to be an open-source project repository that provides a collection of projects across various programming languages and platforms.

# Webpage Image URL Scraping Code:
import requests
from bs4 impor

In [24]:
end_time: float = time.time()
elapsed_time: float = end_time - start_time
elapsed_minutes: int = int(elapsed_time // 60)
elapsed_seconds: float = elapsed_time % 60

logger.info(f"⏱️ Total execution time: {elapsed_minutes}m {elapsed_seconds:.2f}s")
logger.info("✅ Notebook execution completed successfully.")

2025-08-05 20:26:00 - INFO - ⏱️ Total execution time: 4m 4.63s
2025-08-05 20:26:00 - INFO - ✅ Notebook execution completed successfully.


Built with ❤️ using [**HP AI Studio**](https://hp.com/ai-studio).